In [2]:
import pickle
import numpy as np
from qiskit import BasicAer, execute
from qiskit.quantum_info.states.measures import state_fidelity
from scipy.stats import wasserstein_distance

def reverseBits(num,bitSize): 
    binary = bin(num) 
    reverse = binary[-1:1:-1] 
    reverse = reverse + (bitSize - len(reverse))*'0'
    return int(reverse,2)

def simulate_circ(circ, simulator):
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputstate_ordered = [0 for sv in outputstate]
    for i, sv in enumerate(outputstate):
        reverse_i = reverseBits(i,len(circ.qubits))
        outputstate_ordered[reverse_i] = sv
    if simulator == 'sv':
        return outputstate_ordered
    elif simulator == 'prob':
        output_prob = [np.power(np.absolute(x),2) for x in outputstate_ordered]
        return output_prob
    else:
        raise Exception('Illegal simulator')

dirname = './data'
full_circ = pickle.load(open( '%s/full_circ.p'%dirname, 'rb' ))

original_sv = simulate_circ(full_circ,'sv')
reconstructed_sv = pickle.load(open( '%s/reconstructed_sv.p'%dirname, 'rb' ))
fidelity = state_fidelity(original_sv,reconstructed_sv)
print('state vector reconstruction fidelity = ',fidelity)

original_prob = simulate_circ(full_circ,'prob')
reconstructed_prob = pickle.load(open( '%s/reconstructed_prob.p'%dirname, 'rb' ))
distance = wasserstein_distance(original_prob,reconstructed_prob)
print('probability reconstruction distance = ',distance)
# print('first element comparison:',original_prob[0],reconstructed_prob[0])

state vector reconstruction fidelity =  0.9999999999999971
probability reconstruction distance =  4.1914217614249497e-19
